In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import math 
import numpy as np # linear algebra
import h5py
import pandas as pd                                             # data processing, CSV file I/O (e.g. pd.read_csv)
import keras as K
#import tensorflow.compat.v1 as tf                              # tensorflow 2.0 active with keras as the high level api
#tf.disable_v2_behavior()
#import tensorflow as tf
import csv 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [33]:
def input(filename):
   
    with open(filename,'r') as f:
        reader = csv.reader(f)
        l = list(reader)
        
    l.pop(0)
    y = []  
    for i in range(len(l)):
        y.append(l[i][1])
    [j.pop(1) for j in l]
    x = np.array(l)
    
    return x,y



In [34]:
X,Y = input('/kaggle/input/titanic/train.csv')
#my_data = pd.read_csv('')
print("Dimensions of the input are " + str(X.shape))

Dimensions of the input are (891, 11)


In [ ]:
'''
def sigmoid(z):
    
    x = tf.placeholder(tf.float32,name ="X")
    sigmoid = tf.sigmoid(x)
    
    s = tf.Session()
    res = s.run(sigmoid,feed_dict={x:z})
    
    return res
'''

In [ ]:
'''
print ("sigmoid(0) = " + str(sigmoid(0)))
print ("sigmoid(12) = " + str(sigmoid(12)))
'''

In [35]:
parameters = {}

In [39]:
def initialize_parameters():
    
    W1 = tf.get_variable("W1", [25,12288], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [25,1], initializer = tf.zeros_initializer())
    W2 = tf.get_variable("W2", [12,25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [12,1], initializer = tf.zeros_initializer())
    W3 = tf.get_variable("W3", [6,12], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [6,1], initializer = tf.zeros_initializer())
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    return parameters
    

In [9]:
def comp_cost(Y,y_hat):
    
    logits = tf.transpose(y_hat)
    labels = tf.transpose(Y)
    
    z = tf.placeholder(tf.float32,name="z")
    y = tf.placeholder(tf.float32,name="y")
    
    cost = tf.reduce_sumtf.square(y-y-hat)
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))             ### for 
    
    sess = tf.Session()
    cost = sess.run(cost,feed_dict={z:logits,y:labels})
    
    sess.close()
    return cost
    

In [ ]:
def model: 
